In [1]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../utils')
from split_dataset import split_dataframe_by_keys
from dataset_loader import TidySequentialDataCSVLoader
%matplotlib inline

## Load the vitals and demographics

In [2]:
data_dir = '/data/MIMIC-IV/'
suffix = '_first_24_hours'
ts_csv = os.path.join(data_dir, 'features_per_tstep%s_los_prediction.csv.gz'%suffix)
dem_csv = os.path.join(data_dir, 'demographics%s_los_prediction.csv.gz'%suffix)
outcomes_csv = os.path.join(data_dir, 'outcomes_per_seq%s_los_prediction.csv'%suffix)


ts_df = pd.read_csv(ts_csv)
dem_df = pd.read_csv(dem_csv)
outcomes_df = pd.read_csv(outcomes_csv)

## Remove stays less than 30 hours

In [3]:
min_stay_hrs = 30
keep_inds = outcomes_df['length_of_stay_in_hours']>=min_stay_hrs
outcomes_df = outcomes_df.loc[keep_inds, :].copy().reset_index(drop=True)
ts_df = ts_df.loc[ts_df['stay_id'].isin(outcomes_df['stay_id']), :].reset_index(drop=True)
dem_df = dem_df.loc[ts_df['stay_id'].isin(outcomes_df['stay_id']), :].reset_index(drop=True)


stay_lengths = outcomes_df['length_of_stay_in_hours'].values
n_stays = len(outcomes_df['stay_id'].unique())
n_patients = len(outcomes_df['subject_id'].unique())
n_deaths = outcomes_df['in_icu_mortality'].sum()

print('Total stays : %d'%n_stays)
print('Total patients : %d'%n_patients)
print('Frac of stays resulting in death : %.3f'%(n_deaths/n_stays))
print('Frac of patients who die : %.3f'%(n_deaths/n_patients))

for min_los in [3, 7, 11]:
    inds = stay_lengths>=min_los*24
    frac_above_min_los = len(stay_lengths[inds])/n_stays
    print('Frac stays > %d days : %.3f'%(min_los, frac_above_min_los))

Total stays : 52354
Total patients : 38939
Frac of stays resulting in death : 0.081
Frac of patients who die : 0.109
Frac stays > 3 days : 0.466
Frac stays > 7 days : 0.162
Frac stays > 11 days : 0.083


## Get the range of measurements of all features

In [3]:
ts_feature_cols = ['Heart Rate', 
                    'Respiratory Rate', 
                    'O2 saturation pulseoxymetry',
       'Non Invasive Blood Pressure systolic',
       'Non Invasive Blood Pressure diastolic',
        'Temperature Fahrenheit',
        'Height (cm)',
       'Respiratory Rate (Total)', 
       'Potassium (serum)',
       'Sodium (serum)', 
        'Chloride (serum)', 
        'Hematocrit (serum)',
       'Hemoglobin', 
        'Creatinine (serum)', 
        'Glucose (serum)', 
        'Magnesium', 
       'Phosphorous', 
        'Platelet Count', 
        'Glucose (whole blood)',
        'Daily Weight', 
        'Absolute Neutrophil Count',
        'Prothrombin time',
        'Fibrinogen',
        'PH (Arterial)',
        'PH (Venous)',
        'HCO3 (serum)',
        'Arterial O2 pressure',
        'Arterial CO2 Pressure',
        'Lactic Acid',
        'Albumin',
        'Calcium non-ionized',
        'C Reactive Protein (CRP)',
        'ALT',
        'AST',
        'Direct Bilirubin', 
        'Total Bilirubin',
        'Troponin-T',
        'Venous CO2 Pressure']

dem_cols = ['Age', 'is_gender_male', 'is_gender_unknown']
id_col = ['stay_id']
id_cols = ['subject_id', 'hadm_id', 'stay_id']

feature_cols = ts_feature_cols+dem_cols
features_df = pd.merge(ts_df, dem_df, on=id_cols, how='left')

In [ ]:
id_col = ['stay_id']
time_cols = ['hours_from_admission', 'timestamp']

feats_summary_df = pd.DataFrame()
for lab in feature_cols:
    curr_lab_series = features_df[lab]
#     feats_summary_df.loc[lab, 'min'] = curr_lab_series.min()
#     feats_summary_df.loc[lab, 'max'] = curr_lab_series.max()
    feats_summary_df.loc[lab, '10%'] = np.nanpercentile(curr_lab_series, 10)
    feats_summary_df.loc[lab, '90%'] = np.nanpercentile(curr_lab_series, 90)
    feats_summary_df.loc[lab, 'median'] = curr_lab_series.median()


lab_counts_per_stay_df = features_df.groupby(id_col).count()
labs_missing_rate_entire_stay_dict = dict()
for lab in feature_cols:
    labs_missing_rate_entire_stay_dict[lab] = ((lab_counts_per_stay_df[lab]==0).sum())/lab_counts_per_stay_df.shape[0]
labs_missing_rate_entire_stay_series = pd.Series(labs_missing_rate_entire_stay_dict)

feats_summary_df.loc[:,'missing_rate'] = labs_missing_rate_entire_stay_series
feats_summary_df = feats_summary_df[['10%', 'median', '90%', 'missing_rate']]
feats_summary_df.to_csv('ts_feats_summary.csv')
feats_summary_df.round(2)

## Split in train/valid/test

In [6]:
random_state=41
x_train_df, x_test_df = split_dataframe_by_keys(
    features_df, cols_to_group=id_cols, size=0.2, random_state=random_state)

x_train_df, x_valid_df = split_dataframe_by_keys(
        x_train_df, cols_to_group=id_cols, size=0.2, random_state=random_state) 


y_train_df, y_test_df = split_dataframe_by_keys(
    outcomes_df, cols_to_group=id_cols, size=0.2, random_state=random_state)

y_train_df, y_valid_df = split_dataframe_by_keys(
        y_train_df, cols_to_group=id_cols, size=0.2, random_state=random_state) 


## Get the train valid test set stats and save data

In [9]:
for split, y_df, x_df in [('train', y_train_df, x_train_df),
                   ('valid', y_valid_df, x_valid_df),
                   ('test', y_test_df, x_test_df)]:

    stay_lengths = y_df['length_of_stay_in_hours'].values
    n_stays = len(y_df['stay_id'].unique())
    n_patients = len(y_df['subject_id'].unique())
    n_deaths = y_df['in_icu_mortality'].sum()

    print('Total stays : %d'%n_stays)
    print('Total patients : %d'%n_patients)
    print('Frac of stays resulting in death : %.3f'%(n_deaths/n_stays))
    print('Frac of patients who die : %.3f'%(n_deaths/n_patients))
    
    
    save_dir = 'data/MIMIC-IV/mortality_prediction/'
    isExist = os.path.exists(save_dir)

    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(save_dir)
    
    
    for min_los in [3, 7, 11]:
        inds = stay_lengths>=min_los*24
        frac_above_min_los = len(stay_lengths[inds])/n_stays
        print('Frac stays > %d days in %s : %.3f'%(min_los, split, frac_above_min_los))
        y_df['los_geq_%s_days'%min_los] = (stay_lengths>=min_los*24)*1
    
    x_df.to_csv(os.path.join(save_dir, 'x_%s.csv'%split))   
    y_df.to_csv(os.path.join(save_dir, 'y_%s.csv'%split))  

Total stays : 33506
Total patients : 27084
Frac of stays resulting in death : 0.081
Frac of patients who die : 0.100
Frac stays > 3 days in train : 0.467
Frac stays > 7 days in train : 0.161
Frac stays > 11 days in train : 0.082
Total stays : 8377
Total patients : 7821
Frac of stays resulting in death : 0.080
Frac of patients who die : 0.086
Frac stays > 3 days in valid : 0.456
Frac stays > 7 days in valid : 0.160
Frac stays > 11 days in valid : 0.085
Total stays : 10471
Total patients : 9673
Frac of stays resulting in death : 0.084
Frac of patients who die : 0.090
Frac stays > 3 days in test : 0.472
Frac stays > 7 days in test : 0.166
Frac stays > 11 days in test : 0.085


## Save the fully supervised train/valid/test splits as numpy arrays for binary classification

In [12]:
## Convert from dataframe to numpy array of NxTxD
train_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_train_df,
    y_csv_path=y_train_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

valid_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_valid_df,
    y_csv_path=y_valid_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

test_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_test_df,
    y_csv_path=y_test_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

train_x_NTD, y_train = train_vitals.get_batch_data(batch_id=0)
valid_x_NTD, y_valid = valid_vitals.get_batch_data(batch_id=0)
test_x_NTD, y_test = test_vitals.get_batch_data(batch_id=0)

N_tr = len(train_x_NTD)
N_va = len(valid_x_NTD)
N_te = len(test_x_NTD)

print('positive label fraction in train : %.3f'%(y_train.sum()/len(y_train)))
print('positive label fraction in valid : %.3f'%(y_valid.sum()/len(y_valid)))
print('positive label fraction in test : %.3f'%(y_test.sum()/len(y_test)))

## save the data

# curr_save_dir = os.path.join(save_dir, 'los_geq_%s_days_prediction'%min_los)
# # Check whether the specified path exists or not
# isExist = os.path.exists(curr_save_dir)

# if not isExist:
#     # Create a new directory because it does not exist 
#     os.makedirs(curr_save_dir)

# save the data to the respective folder
print('Saving data to %s'%save_dir)
np.save(os.path.join(save_dir, 'X_train.npy'), train_x_NTD)
np.save(os.path.join(save_dir, 'y_train.npy'), y_train)
print('Done saving train..')
np.save(os.path.join(save_dir, 'X_valid.npy'), valid_x_NTD)
np.save(os.path.join(save_dir, 'y_valid.npy'), y_valid)
print('Done saving valid..')
np.save(os.path.join(save_dir, 'X_test.npy'), test_x_NTD)
np.save(os.path.join(save_dir, 'y_test.npy'), y_test)
print('Done saving test..')


positive label fraction in train : 0.081
positive label fraction in valid : 0.080
positive label fraction in test : 0.084
Saving data to /cluster/tufts/hugheslab/datasets/MIMIC-IV/mortality_prediction/
Done saving train..
Done saving valid..
Done saving test..


## Save binary classification data based on % of labeled sequences

In [ ]:
state_id = 41


## Convert from dataframe to numpy array of NxTxD
train_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_train_df,
    y_csv_path=y_train_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

valid_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_valid_df,
    y_csv_path=y_valid_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

test_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_test_df,
    y_csv_path=y_test_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="in_icu_mortality",
    y_label_type='per_sequence'
)

train_x_NTD, y_train = train_vitals.get_batch_data(batch_id=0)
valid_x_NTD, y_valid = valid_vitals.get_batch_data(batch_id=0)
test_x_NTD, y_test = test_vitals.get_batch_data(batch_id=0)

for ii, perc_labelled in enumerate([1.2, 3.7, 11.1, 33.3, 100]):#3.7, 11.1, 33.3, 100
    curr_save_dir = os.path.join(save_dir, 'percentage_labelled_sequences=%s'%perc_labelled)

    print('---------------------------------------------------------------------------')
    print('CREATING TRAIN/VALID/TEST SPLITS FOR %.3f PERCENT OF SEQUENCES LABELLED'%perc_labelled)
    print('---------------------------------------------------------------------------')
    y_train_ss = y_train.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_tr = int((1-(perc_labelled)/100)*N_tr)
    unlabelled_inds_tr = rnd_state.permutation(N_tr)[:n_unlabelled_tr]
    y_train_ss = y_train_ss.astype(np.float32)
    y_train_ss[unlabelled_inds_tr] = np.nan  
    if perc_labelled!=100:
        print('Excluded inds train: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_tr[0],
                                                              unlabelled_inds_tr[1],
                                                              unlabelled_inds_tr[2],
                                                              unlabelled_inds_tr[-3],
                                                              unlabelled_inds_tr[-2],
                                                              unlabelled_inds_tr[-1]))

    y_valid_ss = y_valid.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_va = int((1-(perc_labelled)/100)*N_va)
    unlabelled_inds_va = rnd_state.permutation(N_va)[:n_unlabelled_va]
    y_valid_ss = y_valid_ss.astype(np.float32)
    y_valid_ss[unlabelled_inds_va] = np.nan 
    if perc_labelled!=100:
        print('Excluded inds valid: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_va[0],
                                                          unlabelled_inds_va[1],
                                                          unlabelled_inds_va[2],
                                                          unlabelled_inds_va[-3],
                                                          unlabelled_inds_va[-2],
                                                          unlabelled_inds_va[-1]))

    y_test_ss = y_test.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_te = int((1-(perc_labelled)/100)*N_te)
    unlabelled_inds_te = rnd_state.permutation(N_te)[:n_unlabelled_te]
    y_test_ss = y_test_ss.astype(np.float32)
    y_test_ss[unlabelled_inds_te] = np.nan
    if perc_labelled!=100:
        print('Excluded inds test: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_te[0],
                                                          unlabelled_inds_te[1],
                                                          unlabelled_inds_te[2],
                                                          unlabelled_inds_te[-3],
                                                          unlabelled_inds_te[-2],
                                                          unlabelled_inds_te[-1]))

    # Check whether the specified path exists or not
    isExist = os.path.exists(curr_save_dir)

    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(curr_save_dir)

    # save the data to the respective folder
    print('Saving data to %s'%curr_save_dir)
    np.save(os.path.join(curr_save_dir, 'X_train.npy'), train_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_train.npy'), y_train_ss)
    print('Done saving train..')
    np.save(os.path.join(curr_save_dir, 'X_valid.npy'), valid_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_valid.npy'), y_valid_ss)
    print('Done saving valid..')
    np.save(os.path.join(curr_save_dir, 'X_test.npy'), test_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_test.npy'), y_test_ss)
    print('Done saving test..')


    print('---------------------------------------------------------------------------')
    for split, y in [('train', y_train_ss),
                    ('valid', y_valid_ss),
                    ('test', y_test_ss)]:
        frac_pos_labels = np.nansum(y)/(~np.isnan(y)).sum()
        print('fraction positive labels in %s set with %.3f percent of sequences labelled : %.4f'%(split,
                                                                                                   perc_labelled,
                                                                                                   frac_pos_labels))
    print('---------------------------------------------------------------------------')
    